In [ ]:
# MongoDB 연결

from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from dotenv import load_dotenv
import os

load_dotenv()

USERNAME = os.getenv("MONGO_USERNAME")
PASSWORD = os.getenv("MONGO_PASSWORD")
HOST = os.getenv("MONGO_HOST")
PORT = int(os.getenv("MONGO_PORT"))

url = f"mongodb://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/"


try:
    client = MongoClient(url)
    client.admin.command('ping')
    print("Successfully connected to MongoDB!")

except ConnectionFailure as e:
    print(f"MongoDB connection failed: {e}")

db = client['s307_db']
collection = db['s307_collection']

In [ ]:

import pdfplumber
from PIL import Image

sample_pdf = "gpt-020-3m-sds.pdf"

pdf = pdfplumber.open(sample_pdf)
page_count = len(pdf.pages)

# Root 객체 생성
root = {
    "file_name": sample_pdf,
    "page_num": 0,
    "page_count": page_count,
    "width": pdf.pages[0].width,
    "height": pdf.pages[0].height
}

# Root 객체 추가
root_insert = collection.insert_one(root)
print(f"Inserted Root | File: {root['file_name']} | Page Count: {root['page_count']} | ID: {root_insert.inserted_id}")

In [ ]:
type(pdf.pages[0].images[0]['stream'])

In [ ]:
pdf.pages[0].images[0]['stream'].attrs

In [ ]:
# Image.frombytes('RGB',(67, 36), pdf.pages[0].images[0]['stream'].get_data())

In [ ]:

for i in range(page_count):
    page = pdf.pages[i]

    keys = page.objects.keys()


    images = {}

    if "image" in keys:
        images = page.images
        for image in images:
            raw = image['stream']
            objid = image['stream'].objid
            attrs = image['stream'].attrs
            img = Image.frombytes("RGB" if attrs['ColorSpace'] == 'DeviceRGB' else 'RGB',(attrs['Width'], attrs['Height']), raw.get_data())
            img.show()


    
    # # 키에 존재하는 값만 추가
    # page_objects = {
    #     "file_name": sample_pdf,
    #     "page_num": page.page_number,
    #     "keys": list(page.objects.keys()),
    #     **({"lines": page.lines} if "line" in keys else {}),
    #     **({"rects": page.rects} if "rect" in keys else {}),
    #     **({"chars": page.chars} if "char" in keys else {}),
    #     **({"curves": page.curves} if "curve" in keys else {}),
    #     # **({"images": page.images} if "image" in keys else {})
    # }


    # print(page_objects.keys())
    # result = collection.insert_one(page_objects)
    # print(f"Inserted File: {sample_pdf} | Page: {page.page_number} | ID: {result.inserted_id}")

In [ ]:
pdf.pages[0].page_number

In [ ]:
pdf.pages[0].rects[0]

In [ ]:
pdf.pages[0].chars[0]

In [ ]:
pdf.pages[0].images

In [ ]:
import fitz
import io
from PIL import Image

doc = fitz.open(sample_pdf)

for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    image_list = page.get_images(full=True)

    for img_index, img in enumerate(image_list):
        # 이미지의 고유 식별자(xref) 가져오기
        xref = img[0]

        # 이미지의 기본 정보 추출
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]
        
        # 이미지 확장자 가져오기
        ext = base_image["ext"]
        
        # 이미지 파일로 저장
        image_filename = f"page{page_num+1}_img{img_index}.{ext}"
        with open(image_filename, "wb") as f:
            f.write(image_bytes)
            print(f"'{image_filename}' 파일로 이미지를 저장했습니다.")

doc.close()

In [ ]:
doc = fitz.open(sample_pdf)
doc

In [ ]:
len(doc)

In [ ]:
doc.load_page(0).get_images(full=True)[0]

In [ ]:
pdf.pages[0].images[0]['stream']

In [ ]:
doc.extract_image(48)

In [ ]:
from PIL import Image

Image.frombytes('RGB',(67, 36), pdf.pages[0].images[0]['stream'].get_data())

In [ ]:
collection.find_one({"file_name": "gpt-020-3m-sds.pdf", "page_num": 1})['chars']

In [ ]:
type(pdf.pages[0].images)

In [ ]:
image_objects = []

page_num = pdf.pages[0].page_number

for image_index, image in enumerate(pdf.pages[0].images):
    # 이미지 정보 추출
    raw = image['stream']
    attrs = image['stream'].attrs
    if attrs['ColorSpace'] == '/DeviceRGB':
        color_space = 'RGB'
    else:
        color_space = 'RGB'

    # 이미지 생성
    img = Image.frombytes(color_space, (attrs['Width'], attrs['Height']), raw.get_data())
    img_url = f"storage/page{page_num}_img{image_index}.png"
    img.save(img_url)

    # 이미지 정보 수정
    del image['stream']
    image['img_url'] = img_url

    # 이미지 객체 추가
    image_objects.append(image)

print(image_objects)

In [ ]:
# 작업 종료

pdf.close()
client.close()

In [ ]:
import pdfplumber

pdf_path = "gpt-020-3m-sds.pdf"
pdf = pdfplumber.open(sample_pdf)
storage_path = "storage"

raw = pdf.pages[0].images[0]['stream']

In [ ]:
test = raw.get_data()

In [ ]:
import pdfplumber

pdf_path = "gpt-020-3m-sds.pdf"
pdf = pdfplumber.open(sample_pdf)
storage_path = "storage"

# Root Object 생성
root_object = {
    "file_name": pdf_path.split("/")[-1],
    "page_num": 0,
    "page_count": len(pdf.pages),
    "width": pdf.pages[0].width,
    "height": pdf.pages[0].height
}

for page_idx in range(len(pdf.pages)):

    page = pdf.pages[page_idx]
    keys = list(page.objects.keys())


    if "image" in keys:
        image_objects = []
        previous_image_objects = page.images.copy()
        for image_index, image in enumerate(previous_image_objects):
            raw = image['stream']
            attrs = image['stream'].attrs
            img = Image.frombytes("RGB", (attrs['Width'], attrs['Height']), raw.get_data())
            img_url = f"storage/page{page_idx+1}_img{image_index}.png"
            img.save(img_url)

            del image['stream']
            image['img_url'] = img_url

            image_objects.append(image)



    page_object = {
        "file_name": pdf_path.split("/")[-1],
        "page_num": page.page_number,
        "keys": keys,
        **({"lines": page.lines} if "line" in keys else {}),
        **({"rects": page.rects} if "rect" in keys else {}),
        **({"chars": page.chars} if "char" in keys else {}),
        **({"curves": page.curves} if "curve" in keys else {}),
        **({"images": image_objects} if "image" in keys else {})
    }

    print(page_object.keys())

In [ ]:
# 원본 코드
from PIL import Image
from io import BytesIO

# 직렬화 되지 않는 이미지 정보 처리 + 이미지 저장 + url 추가
def make_image_doc_serializable(page_num: int, img: dict) -> dict:
    # 이미지 폴더 생성 (없으면 자동 생성)
    os.makedirs("images", exist_ok=True)

    # png 제작을 위한 이미지 정보 추출
    img_id = str(img['name'])
    img_data = img['stream'].get_data()    # 바이트
    img_info = img['stream'].attrs
    img_width = int(img_info['Width'])
    img_height = int(img_info['Height'])
    img_bits = int(img_info['BitsPerComponent'])

    img_colorspace = str(img_info['ColorSpace']).lstrip('/').upper()
    print(img_colorspace[:10])
    img_filter_raw = img_info.get('Filter')
    img_filters = (
        [] if img_filter_raw is None
        else ([str(x).lstrip('/').strip(" '\"").upper() for x in img_filter_raw]
            if isinstance(img_filter_raw, (list, tuple))
            else [str(img_filter_raw).lstrip('/').strip(" '\"").upper()])
    )
    has_jpeg_like = any(f in ('DCTDECODE', 'JPXDECODE') for f in img_filters)

    # 이미지 모드 설정
    if img_colorspace in ('RGB', 'DEVICERGB'):
        img_mode = 'RGB'
    elif img_colorspace in ('GRAY', 'DEVICEGRAY'):
        img_mode = '1' if img_bits == 1 else 'L'
    elif img_colorspace in ('CMYK', 'DEVICECMYK'):
        img_mode = 'CMYK'
    else:
        img_mode = 'RGB'
        
    img_size = (img_width, img_height)



    # filter별 이미지 처리
    if has_jpeg_like:
        print("jpeg 이미지 발견\n")
        try:
            pil_image = Image.open(BytesIO(img_data))
            pil_image = pil_image.convert('RGB')
            pil_image.save(f"images/{sample_pdf}_{page_num}_{img_id}.png")
            img_url = str(f"images/{sample_pdf}_{page_num}_{img_id}.png")
            print(f"{img_id} 이미지 저장 완료\n")
        except Exception as e:
            print(f"이미지 저장 오류: {e}")
            img_url = None
    else:
        print("그 외 이미지 발견\n")
        try:
            pil_image = Image.frombytes(img_mode, img_size, img_data)
            pil_image = pil_image.convert('RGB')
            pil_image.save(f"images/{sample_pdf}_{page_num}_{img_id}.png")
            img_url = str(f"images/{sample_pdf}_{page_num}_{img_id}.png")
            print(f"{img_id} 이미지 저장 완료\n")
        except Exception as e:
            print(f"이미지 저장 오류: {e}")
            img_url = None
        
    
    out = {k: v for k, v in img.items() if k != "stream"}  # stream 제거
    if "colorspace" in out:
        out["colorspace"] = [str(cs).lstrip("/") for cs in out["colorspace"]]
    if isinstance(out.get("srcsize"), tuple):
        out["srcsize"] = list(out["srcsize"])

    out['url'] = img_url
    return out
import pdfplumber

pdf_path = "gpt-020-3m-sds.pdf"
pdf = pdfplumber.open(sample_pdf)
storage_path = "storage"

# Root Object 생성
root_object = {
    "file_name": pdf_path.split("/")[-1],
    "page_num": 0,
    "page_count": len(pdf.pages),
    "width": pdf.pages[0].width,
    "height": pdf.pages[0].height
}

page_idx = 0

page = pdf.pages[page_idx]
keys = list(page.objects.keys())


if "image" in keys:
    image_objects = []
    previous_image_objects = page.images.copy()
    for image in previous_image_objects:
        new_image_object = make_image_doc_serializable(page.page_number, image)
        image_objects.append(new_image_object)



page_object = {
    "file_name": pdf_path.split("/")[-1],
    "page_num": page.page_number,
    "keys": keys,
    **({"lines": page.lines} if "line" in keys else {}),
    **({"rects": page.rects} if "rect" in keys else {}),
    **({"chars": page.chars} if "char" in keys else {}),
    **({"curves": page.curves} if "curve" in keys else {}),
    **({"images": image_objects} if "image" in keys else {})
}

print('image objects: ', page_object["images"])

In [ ]:
# MongoDB 연결

from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from dotenv import load_dotenv
import os

load_dotenv()

USERNAME = os.getenv("MONGO_USERNAME")
PASSWORD = os.getenv("MONGO_PASSWORD")
HOST = os.getenv("MONGO_HOST")
PORT = int(os.getenv("MONGO_PORT"))

url = f"mongodb://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/"


try:
    client = MongoClient(url)
    client.admin.command('ping')
    print("Successfully connected to MongoDB!")

except ConnectionFailure as e:
    print(f"MongoDB connection failed: {e}")

db = client['s307_db']
collection = db['s307_collection']

In [ ]:
# 기존 코드
# collection.insert_one(page_object)

# 수정된 코드: "Upsert" 로직 적용
filter_query = {
    "file_name": page_object["file_name"], 
    "page_num": page_object["page_num"]
}
collection.replace_one(filter_query, page_object, upsert=True)

print("Upsert operation completed.")

# 확인 (동일한 쿼리로 조회)
found_doc = collection.find_one(filter_query)
if found_doc:
    print("Found document keys:", found_doc.keys())
else:
    print("Document not found.")

In [ ]:
collection.find_one({"file_name": "gpt-020-3m-sds.pdf", "page_num": 1}).keys()